In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne,os,glob                    
from mne.io import read_raw_ctf
from pathlib import Path
from sklearn import mixture
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, normalized_mutual_info_score

# Globus_raw_Data

In [ ]:

rootdir = '/media/ubuntu/HDD_P2/MDD/1sttrial/*'
training_data = np.empty((0,271,3000))
training_data1 = np.empty((0,271,3000))
i=0
for path in glob.glob(rootdir):
    if path.endswith('.ds'):
        i+=1
        # read .ds file
        raw = read_raw_ctf(path, preload=True)   
        # resampling from 1200 to 300Hz 
        raw.resample(300).pick('meg').load_data()
        # make 10s epochs from continuous data 
        duration = 10.
        events = mne.make_fixed_length_events(raw, duration=duration)
        tmax = duration - 1. / raw.info['sfreq']
        # picks = mne.pick_types(raw.info, meg=True,ref_meg=False)
        epochs = mne.Epochs(raw, events=events, tmin=0, tmax=tmax,baseline=None)

        epoch1 = epochs.load_data()
        # picking onlu meg channels
        epoch2 = epochs.pick_types(meg=True, ref_meg=False).load_data()
       # print(epochs.ch_names)
        # dropping extra channels
        if len(epochs.ch_names)==272:
            epochs.drop_channels(['MLT31-1609'])
        if len(epochs.ch_names)==273:
            epochs.drop_channels(['MLT31-1609'])
            epochs.drop_channels(['MLF25-1609'])
        epoch4 = epochs.get_data()
        print(np.shape(epoch4))
        epochs.load_data().filter(l_freq=4, h_freq=13)
        picks = mne.pick_types(raw.info, meg=True, ref_meg=False)
        alpha_data = epochs.get_data()
        #l1 = alpha_data.shape[0]
        # concatenate unfiltered data 
        training_data=np.concatenate((training_data,epoch4[:,:,:]),axis=0)
        # concatenate filtered data 
        training_data1=np.concatenate((training_data1,alpha_data[:,:,:]),axis=0)
print(i)


In [ ]:
# saving filtered and unfiltered data 
np.savez('/media/ubuntu/HDD_P2/MDD/globus_data2.npz', data=training_data, data1 = training_data1, labels = labels)

# Bids_mdd sample data with labels 

In [ ]:
#Reading labels for bids_mdd sample resting state datatest for supervised learning 
df = pd.read_excel('/home/ubuntu/Downloads/bids_lab.xlsx')
print(df.head())
df1 = df['labels'].values
print(df1[0])
print(len(df1))

In [ ]:
#loading BIDS data 
blabels=[]
lab=[]
la=[]

j=0
rootdir = '/media/ubuntu/HDD_P2/MDD/DATA'
btraining_data = np.empty((0,271,3000))
btraining_data1 = np.empty((0,271,3000))
for path in glob.glob(f'{rootdir}/*/'):
    path1 = os.path.join(path,'meg/')
#    print(path1)

    for raw_path, i  in zip(glob.glob(f'{path1}/*'),range(44)):
        if raw_path.endswith('_task-rest_run-1_meg.ds'):
            # loading bids_data label
            l = df1[i]
            la.append(l)
            j +=1
            #read bids_sample data .ds file
            raw = read_raw_ctf(raw_path, preload=True)   
            # resampling from 1200 to 300Hz
            raw.resample(300).load_data()
            # epoching into 10s trial
            duration = 10.
            events = mne.make_fixed_length_events(raw, duration=duration)
            tmax = duration - 1. / raw.info['sfreq']
            picks = mne.pick_types(raw.info, meg=True,ref_meg=False)
            epochs = mne.Epochs(raw, events=events, tmin=0, tmax=tmax,
                                baseline=None)
            sfreq = epochs.info['sfreq']
            n_time_samps = raw.n_times
            time_secs = raw.times
            ch_names = raw.ch_names
            n_chan = len(ch_names)  # note: there is no raw.n_channels attribute
            print('the (cropped) sample data object has {} time samples and {} channels.'
                  ''.format(n_time_samps, n_chan))
            print('The last time sample is at {} seconds.'.format(time_secs[-1]))
            print('The first few channel names are {}.'.format(', '.join(ch_names[:3])))
            print()  # insert a blank line in the output

            # some examples of raw.info:
            print('bad channels:', raw.info['bads'])  # chs marked "bad" during acquisition
            print(raw.info['sfreq'], 'Hz')            # sampling frequency
            print(raw.info['description'], '\n')      # miscellaneous acquisition info

            print(raw.info)
            
            epoch1 = epochs.load_data()
            #selecting meggrad channels
            epoch3 = epochs.pick_types(meg=True, ref_meg=False).load_data()
            #removing channel which is not in competition dataset 
            if len(epochs.ch_names)==272:
                epochs.drop_channels(['MLT31-1609'])
            if len(epochs.ch_names)==273:
                epochs.drop_channels(['MLT31-1609'],['MLF25-1609'])
            
            epoch4 = epochs.get_data()
            print(np.shape(epoch4))
            #filtering data in alpha band 
            epochs.load_data().filter(l_freq=4, h_freq=13)
            picks = mne.pick_types(raw.info, meg=True, ref_meg=False)
            alpha_data = epochs.get_data()
            #appending labels based on number of trials for each subject  # nonuniform trials
            l1 = alpha_data.shape[0]
            lab.append(l1)
            for i in range(l1):
                blabels.append(l)
            #unfiltered data 
            btraining_data=np.concatenate((btraining_data,epoch4[:,:,:]),axis=0)
            #filtered data
            btraining_data1=np.concatenate((btraining_data1,alpha_data[:,:,:]),axis=0)
        
print('j=',j)

#tf1 = pd.DataFrame(la).to_csv('/media/ubuntu/HDD_P2/la.csv')

In [ ]:
#saving bids_mdd filtered and unfiltered data 
np.savez('/media/ubuntu/HDD_P2/MDD/bids_data3.npz', data=btraining_data, data1 = btraining_data1, blabels = blabels)

In [ ]:
#loading bids unfiltered data 
data = btraining_data

# splitting, covariance matrix , channel selection, Tangent space 

In [ ]:
#splitting bids_data into train and validation set using kfold 

from sklearn.model_selection import KFold
from pyriemann.estimation import Covariances
from pyriemann.channelselection import ElectrodeSelection
from pyriemann.tangentspace import TangentSpace
kfold = KFold(n_splits=3, shuffle=True,random_state=10)
## enumeration of the splits
t=[];v=[];t1=[];v1=[];t2=[];v2=[];l1=[];l2=[]
mat=[];channel=[]
for (train, val) in kfold.split(data):
    print('train: %s, val: %s' % (data[train].shape, data[val].shape))
    print('trainl: %s, vall: %s' % (label[train].shape, label[val].shape))
    l1.append(label[train]);l2.append(label[val])
    
    #Calculating pyriemann covariance matrix as feature 
    newmat = Covariances(estimator='lwf').fit(data[train], label[val])                                         
    mat.append(newmat)

    new_train = newmat.transform(data[train])
    t.append(new_train)
    cov_val = newmat.transform(data[val])
    v.append(cov_val)
    
    #Feature selection
    channel_train = ElectrodeSelection(nelec =24).fit(new_train, label[train], sample_weight=None)
    channel.append(channel_train)
   
    
    select_train = channel_train.transform(new_train)
    t1.append(select_train)   
    #Projecting train data on tangent space for feeding into model 
    select_t1 = TangentSpace().fit_transform(select_train)
    t2.append(select_t1)
   
    #feature selection for validation data
    select_val = channel_train.transform(cov_val)
    v1.append(select_val)
    #Projecting validation data on tangent space for feeding into model 
    select_v1 = TangentSpace().fit_transform(select_val)
    v2.append(select_v1)

# Covarinace feature for globus test data 

In [ ]:
#Loading competition test data (Globus)
gdata = np.load('/media/ubuntu/HDD_P2/MDD/globus_data2.npz')
g_data = gdata['data']

#covatiance matrix, channel selection, tangent space transformation
cov_test = mat[2].transform(g_data)
print('shape of covariance matrix=', cov_test.shape)
globus_test = channel_train.transform(cov_test)
ts_test = TangentSpace().fit_transform(globus_test)


In [ ]:
np.savez('riema_test.npz', cov_test=cov_test, ts_test=ts_test)

# kMeans

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 2, n_init = 40)
km.fit(ts_test)

In [ ]:
km.labels_

In [ ]:
pd.DataFrame(km.labels_).to_csv('/media/ubuntu/HDD_P2/MDD/kmeans.csv')

In [ ]:
from collections import Counter
m2 =[]
for i in range(36):
    l1 = km.labels_[((26*i)-(i-1))-1:(25*(i+1)+1)-1]
    #print((range(((26*i)-(i-1))-1,(25*(i+1)+1)-1)))
    #print(l.shape)
    most_common1,num_most_common1 = Counter(l1).most_common(1)[0] # 4, 6 times
    #print(most_common, num_most_common)
    m2.append(most_common1)
tf1 = pd.DataFrame(m2).to_csv('/media/ubuntu/HDD_P2/MDD/final2/kmeans.csv')

In [ ]:
list(m2).count(0)

In [ ]:
m2.count(1)

# GMM predictions

In [3]:
from sklearn import mixture
gmm = mixture.GaussianMixture(n_components=2).fit(ts_test)
labels = gmm.predict(ts_test)


In [ ]:
pd.DataFrame(labels).to_csv('/media/ubuntu/HDD_P2/MDD/gmm_globus.csv')

In [31]:
df = pd.read_csv('/media/ubuntu/HDD_P2/MDD/gmm_globus.csv')
print(df.head())
df1 = df['B'].values
print(df1[0])
print(len(df1))
from collections import Counter
m2 =[]
for i in range(36):
    l1 = df1[((26*i)-(i-1))-1:(25*(i+1)+1)-1]
    #print((range(((26*i)-(i-1))-1,(25*(i+1)+1)-1)))
    #print(l.shape)
    most_common1,num_most_common1 = Counter(l1).most_common(1)[0] # 4, 6 times
    #print(most_common, num_most_common)
    m2.append(most_common1)
tf1 = pd.DataFrame(m2).to_csv('/media/ubuntu/HDD_P2/MDD/final2/gmm.csv')


   A  B
0  0  1
1  1  1
2  2  1
3  3  1
4  4  1
1
900
   Id  GMM
0   1    0
1   2    1
2   3    0
3   4    1
4   5    1
0
36


# GMM + kMeans

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# Initialize KMeans and GaussianMixture models
kmeans = KMeans(n_clusters=2, max_iter=1000,tol=1e-4)

gm = GaussianMixture(n_components=2, max_iter=1000, tol=1e-4,init_params='random')

# Fit and predict the algorithms
y_kmeans = kmeans.fit_predict(test_data)
y_gm = gm.fit_predict(test_data)
y_gm_proba = gm.predict_proba(test_data)

In [ ]:
y_gm_proba.round()

In [ ]:
gm = GaussianMixture(n_components=2, max_iter=3000, tol=1e-4,init_params='random')

gm_kmeans = GaussianMixture(n_components=2,  max_iter=3000, tol=1e-4,init_params='kmeans')

y_gm = gm.fit_predict(test_data)
y_gm_proba = gm.predict_proba(test_data)

y_gm_kmeans = gm_kmeans.fit_predict(test_data)
y_gm_proba_kmeans = gm_kmeans.predict_proba(test_data)

#plt.plot_different_gms(train_data, 0.34,  y_gm, y_gm_proba,  y_gm_kmeans, y_gm_proba_kmeans)

In [ ]:
#y_gm_proba_kmeans[:,0].round()
pred4 = [int(x) for x in y_gm_proba_kmeans[:,0].round()]
np.shape(pred4)

In [ ]:
t1 = pd.DataFrame(pred4).to_csv('/media/ubuntu/HDD_P2/MDD/final2/globus.csv')

In [ ]:
np.shape(pred4)

# counter globus test data

In [ ]:
from collections import Counter
m2 =[]
for i in range(36):
    l1 = pred4[((26*i)-(i-1))-1:(25*(i+1)+1)-1]
    #print((range(((26*i)-(i-1))-1,(25*(i+1)+1)-1)))
    #print(l.shape)
    most_common1,num_most_common1 = Counter(l1).most_common(1)[0] # 4, 6 times
    #print(most_common, num_most_common)
    m2.append(most_common1)
tf1 = pd.DataFrame(m2).to_csv('/media/ubuntu/HDD_P2/MDD/final2/globus_GMM+kMeans.csv')

# prediction counter for Bids_mdd data 

In [ ]:
#counting final label for each subject based on maximum number of occurence 
from collections import Counter
m3 =[]
sum = 0
for i1 in range(0,40):
    #print('i=',i1)
    l2= f1[i1]
    sum = sum+l2
    #print('sum=', sum)
    #print('l2=', l2)
    #print('i=',f1[i],'i+1=',f1[i+1])
    l1 = train_label[sum : sum+f1[i1+1]]
    print(range(sum, sum+f1[i1+1]))
    #print(l2+((((((f1+1)[i]*i+1)-(i))-1))))
    #l1 = pred2[(((((f1+1)[i])*i+1)-(i))-1) :((f1[i])*(i+1)+1)-1]
    #print((range(l2+((((f1+1)[i]*i+1)-(i))-1),((f1[i])*(i+2)+1)-1)))
    #print(l1.shape)
    most_common3,num_most_common3 = Counter(l1).most_common(1)[0] # 4, 6 times
    #print(most_common3, num_most_common3)
    m3.append(most_common3)
tf1 = pd.DataFrame(m3).to_csv('/media/ubuntu/HDD_P2/MDD/final2/bids_GMM+kMeans.csv')

In [ ]:
m3.count(1)